## Imports & Project Setup
✔ Sets project root
✔ Enables imports
✔ Confirms previous notebooks produced outputs
✔ Fails early if something is missing

In [1]:
# ========== BASIC IMPORTS ==========
import sys
from pathlib import Path
import yaml
import pandas as pd
import numpy as np

# ========== PROJECT ROOT ==========
PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

print("Project root:", PROJECT_ROOT)
print("Results folder exists:",
      (PROJECT_ROOT / "results").exists())


Project root: C:\Project_Final_Year
Results folder exists: True


## Load Config
✔ Reads fusion logic from config
✔ Allows tuning without code changes
✔ Keeps SLDCE configurable

In [2]:
# ========== LOAD CONFIG ==========
with open(PROJECT_ROOT / "config/default.yaml", "r") as f:
    config = yaml.safe_load(f)

CONF_W = config["fusion"]["confidence_weight"]
ANOM_W = config["fusion"]["anomaly_weight"]

print("Confidence weight:", CONF_W)
print("Anomaly weight:", ANOM_W)
print("Weight sum:", CONF_W + ANOM_W)


Confidence weight: 0.6
Anomaly weight: 0.4
Weight sum: 1.0


## Load Signal Outputs
✔ Loads outputs from Notebook 03 & 04
✔ Keeps notebooks decoupled
✔ Prepares for fusion

In [5]:
# ========== LOAD SIGNAL FILES ==========
confidence_path = PROJECT_ROOT / "results/confidence_flags.csv"
anomaly_path = PROJECT_ROOT / "results/anomaly_results.csv"

confidence_df = pd.read_csv(confidence_path)
anomaly_df = pd.read_csv(anomaly_path)

print("Confidence signals shape:", confidence_df.shape)
print("Anomaly signals shape:", anomaly_df.shape)

confidence_df.head()


Confidence signals shape: (6513, 112)
Anomaly signals shape: (6513, 110)


,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,label,predicted_label,given_label_confidence,predicted_label_confidence,confidence_flag
0,-0.772657,0.138566,1.132737,-0.146613,-0.217898,0.371553,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,<=50K,<=50K,0.91,0.91,False
1,-1.433042,-0.678059,-0.419226,-0.146613,-0.217898,-1.981349,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,<=50K,<=50K,1.00,1.00,False
2,0.621488,0.205797,1.132737,-0.146613,-0.217898,0.371553,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,>50K,>50K,0.91,0.91,False
3,1.135121,-0.501324,2.296709,-0.146613,-0.217898,-0.034120,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,>50K,>50K,0.58,0.58,False
4,0.621488,1.372661,-0.419226,-0.146613,-0.217898,-0.034120,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,<=50K,<=50K,0.97,0.97,False


## Align & Validate Signals
✔ Prevents silent misalignment bugs
✔ Ensures correct signal fusion
✔ Critical for trustworthy correction decisions

In SLDCE, misaligned signals = wrong decisions, so this check is mandatory

In [6]:
# ========== BASIC VALIDATION ==========
assert len(confidence_df) == len(anomaly_df), \
    "Mismatch between confidence and anomaly signal lengths"

print("Signal alignment check passed")

# We will build combined results on top of confidence_df
combined_df = confidence_df.copy()


Signal alignment check passed


## Normalize Individual Signals (Risk Scores)
We convert different signals into comparable [0,1] risk values.

✔ Converts heterogeneous signals into a common scale
✔ Enables weighted fusion
✔ Keeps logic model-agnostic & dataset-agnostic

You can explain this as:

“Before combining signals, SLDCE normalizes each signal into a unified risk representation, ensuring fair contribution during fusion.”

In [8]:
# ========== CONFIDENCE RISK ==========
# Binary signal → already normalized
combined_df["confidence_risk"] = combined_df["confidence_flag"].astype(int)

# ========== ANOMALY RISK ==========
# Higher anomaly_score = more anomalous
anom_scores = anomaly_df["anomaly_score"]

# Min–max normalization
anom_norm = (anom_scores - anom_scores.min()) / (
    anom_scores.max() - anom_scores.min() + 1e-8
)

combined_df["anomaly_risk"] = anom_norm

print("Confidence risk stats:")
print(combined_df["confidence_risk"].value_counts())

print("\nAnomaly risk stats:")
print(combined_df["anomaly_risk"].describe())


Confidence risk stats:
confidence_risk
0    5540
1     973
Name: count, dtype: int64

Anomaly risk stats:
count    6513.000000
mean        0.271197
std         0.158848
min         0.000000
25%         0.154199
50%         0.248319
75%         0.370650
max         1.000000
Name: anomaly_risk, dtype: float64


## Weighted Signal Fusion (CORE LOGIC)
✔ Combines independent signals
✔ Produces a continuous risk score
✔ Fully config-driven
✔ Model & dataset agnostic

In [9]:
# ========== WEIGHTED SIGNAL FUSION ==========
combined_df["combined_risk_score"] = (
    CONF_W * combined_df["confidence_risk"] +
    ANOM_W * combined_df["anomaly_risk"]
)

print("Combined risk score statistics:")
print(combined_df["combined_risk_score"].describe())


Combined risk score statistics:
count    6513.000000
mean        0.198115
std         0.214726
min         0.000226
25%         0.071975
50%         0.117661
75%         0.192348
max         0.912427
Name: combined_risk_score, dtype: float64


## Save Combined Signals
results/

   └── combined_signals.csv
This file contains:
confidence_flag
confidence_risk
anomaly_risk
combined_risk_score ← 🔑 key output of Phase 5

In [10]:
# ========== SAVE COMBINED SIGNALS ==========
results_path = PROJECT_ROOT / "results"
results_path.mkdir(parents=True, exist_ok=True)

output_path = results_path / "combined_signals.csv"
combined_df.to_csv(output_path, index=False)

print("Combined signals saved successfully")
print("Saved at:", output_path)


Combined signals saved successfully
Saved at: C:\Project_Final_Year\results\combined_signals.csv
